In [119]:
from collections import defaultdict
from copy import copy, deepcopy
import time
start_time = time.time()

class Graph:
    def __init__(self):
        self.V = []
        self.graph = defaultdict(list)
        self.graphsummary = [[],[]]
    def addEdge(self, u, v, w):
        if u not in self.V:
            self.V.append(u)
        if v not in self.V:
            self.V.append(v)
        if u not in self.graphsummary[0]:
            self.graphsummary[0].append(u)
        if v not in self.graphsummary[0]:
            self.graphsummary[0].append(v)
        self.graphsummary[1].append([u,v,w])
        self.graph[v].append(u)
        self.graph[u].append(v)
    def isCyclicUtil(self, v, visited, parent):
        visited[v] = True
        for i in self.graph[v]:
            if visited[i] == False:
                if(self.isCyclicUtil(i, visited, v)):
                    return True
            elif parent != i:
                return True
        return False
        
    def isCyclic(self):
        visited = {}
        for i in self.V:
            visited[i] = False
        for i in self.V:
            if visited[i] == False:
                if(self.isCyclicUtil(i, visited, -1)) == True:
                    return True
        return False
    def contains(self,terminals):
        for i in terminals:
            if i not in self.graphsummary[0]:
                return False
        return True
    def isConnected(self):
        visited = []
        curr = self.V[0]
        queue = []
        for i in self.graph[curr]:
            queue.append(i)
        visited.append(curr)
        while queue:
            curr = queue.pop()
            if curr not in visited:
                visited.append(curr)
                for i in self.graph[curr]:
                    if i not in queue and i not in visited:
                        queue.append(i)
        for i in self.V:
            if i not in visited:
                return False
        return True
best_tree = Graph()
best_cost = 10**1000
def BranchAndBound(G, ST):
    global best_tree
    global best_cost
    BranchAndBoundHelper(G, ST, Graph(), 0)

def BranchAndBoundHelper(G, ST, CT, cost):
    global best_tree
    global best_cost
    if CT.contains(ST) and CT.isConnected():
        if cost < best_cost:
            best_cost = cost
            best_tree = deepcopy(CT)
    else:
        for i in G:
            x = deepcopy(CT)
            x.addEdge(i[0],i[1],i[2])
            if not x.isCyclic():
                BranchAndBoundHelper(G, ST, x, cost + i[2])


G = [[0, 1, 10],[1, 2, 12],[2, 3, 10],[3, 4, 5],[4,5,3],[5,0,7],[0,2,9],[0,3,15],[3,5,1]]
ST = [0, 3 ,4]
BranchAndBound(G, ST)
x = 0
print("Steiner Tree using Branch and Bound Algorithm:")
print("Edges of the tree with corresponding weight")
for i in best_tree.graphsummary[1]:
    print("Edge:",i[0],"-",i[1],"| Weight:",i[2])
    x += i[2]
print("Nodes in final tree",best_tree.V)
print("Minimum Cost",x)
print("Time taken to generate the Steiner Tree: %s seconds" % (time.time() - start_time))

Steiner Tree using Branch and Bound Algorithm:
Edges of the tree with corresponding weight
Edge: 4 - 5 | Weight: 3
Edge: 5 - 0 | Weight: 7
Edge: 3 - 5 | Weight: 1
Nodes in final tree [4, 5, 0, 3]
Minimum Cost 11
Time taken to generate the Steiner Tree: 0.332050085067749 seconds


In [120]:
import time
start_time = time.time()
def floyd_warshall(nodes, edges):
    dist = {node: {node: float('inf') for node in nodes} for node in nodes}
    for node in nodes:
        dist[node][node] = 0
    for u, v, w in edges:
        dist[u][v] = w
        dist[v][u] = w
    
    for k in nodes:
        for i in nodes:
            for j in nodes:
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

def find(parent, i):
    if parent[i] == i:
        return i
    return find(parent, parent[i])

def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)
    if rank[xroot] < rank[yroot]:
        parent[xroot] = yroot
    elif rank[xroot] > rank[yroot]:
        parent[yroot] = xroot
    else:
        parent[yroot] = xroot
        rank[xroot] += 1

def kruskal(nodes, edges):
    parent = {node: node for node in nodes}
    rank = {node: 0 for node in nodes}
    mst_edges = []
    
    edges.sort(key=lambda item: item[2])
    for edge in edges:
        u, v, w = edge
        if find(parent, u) != find(parent, v):
            union(parent, rank, u, v)
            mst_edges.append(edge)
    return mst_edges

nodes = ['0', '1', '2', '3', '4','5']
edges = [('0', '1', 10),('1', '2', 12),('2', '3', 10),('3', '4', 5),('4','5',3),('5','0',7),('0','2',9),('0','3',15),('3','5',1)]
terminals = ['0', '3', '4']

all_pairs_shortest = floyd_warshall(nodes, edges)

complete_edges = [(u, v, all_pairs_shortest[u][v]) for u in terminals for v in terminals if u != v]

mst_edges = kruskal(terminals, complete_edges)

print("Steiner Tree using Vazirani and Yannakakis Algorithm:")
print("Edges of the tree with corresponding weight")
total_weight = 0
node = []
for u, v, w in mst_edges:
    if u not in node:
        node.append(u)
    if v not in node:
        node.append(v)
    print(f"Edge: {u} - {v} | Weight: {w}")
    total_weight += w
print("Nodes in final tree",node)
print("Minimum Cost",total_weight)
print("Time taken to generate the Steiner Tree: %s seconds" % (time.time() - start_time))

Steiner Tree using Vazirani and Yannakakis Algorithm:
Edges of the tree with corresponding weight
Edge: 3 - 4 | Weight: 4
Edge: 0 - 3 | Weight: 8
Nodes in final tree ['3', '4', '0']
Minimum Cost 12
Time taken to generate the Steiner Tree: 0.0018329620361328125 seconds


In [121]:
from collections import defaultdict
from copy import copy, deepcopy
import time
start_time = time.time()

class Graph:
    def __init__(self):
        self.V = []
        self.graph = defaultdict(list)
        self.graphsummary = [[],[]]
    def addEdge(self, u, v, w):
        if u not in self.V:
            self.V.append(u)
        if v not in self.V:
            self.V.append(v)
        if u not in self.graphsummary[0]:
            self.graphsummary[0].append(u)
        if v not in self.graphsummary[0]:
            self.graphsummary[0].append(v)
        self.graphsummary[1].append([u,v,w])
        self.graph[v].append(u)
        self.graph[u].append(v)
    def isCyclicUtil(self, v, visited, parent):
        visited[v] = True
        for i in self.graph[v]:
            if visited[i] == False:
                if(self.isCyclicUtil(i, visited, v)):
                    return True
            elif parent != i:
                return True
        return False
        
    def isCyclic(self):
        visited = {}
        for i in self.V:
            visited[i] = False
        for i in self.V:
            if visited[i] == False:
                if(self.isCyclicUtil(i, visited, -1)) == True:
                    return True
        return False
    def contains(self,terminals):
        for i in terminals:
            if i not in self.graphsummary[0]:
                return False
        return True
    def isConnected(self):
        visited = []
        curr = self.V[0]
        queue = []
        for i in self.graph[curr]:
            queue.append(i)
        visited.append(curr)
        while queue:
            curr = queue.pop()
            if curr not in visited:
                visited.append(curr)
                for i in self.graph[curr]:
                    if i not in queue and i not in visited:
                        queue.append(i)
        for i in self.V:
            if i not in visited:
                return False
        return True
best_tree = Graph()
best_cost = 10**1000
def BranchAndBound(G, ST):
    global best_tree
    global best_cost
    BranchAndBoundHelper(G, ST, Graph(), 0)

def BranchAndBoundHelper(G, ST, CT, cost):
    global best_tree
    global best_cost
    if CT.contains(ST) and CT.isConnected():
        if cost < best_cost:
            best_cost = cost
            best_tree = deepcopy(CT)
    else:
        for i in G:
            x = deepcopy(CT)
            x.addEdge(i[0],i[1],i[2])
            if not x.isCyclic() and cost + i[2] < best_cost:
                BranchAndBoundHelper(G, ST, x, cost + i[2])


G = [[0, 1, 10],[1, 2, 12],[2, 3, 10],[3, 4, 5],[4,5,3],[5,0,7],[0,2,9],[0,3,15],[3,5,1]]
ST = [0, 3 ,4]
BranchAndBound(G, ST)
x = 0
print("Steiner Tree using Branch and Bound Algorithm with pruning:")
print("Edges of the tree with corresponding weight")
for i in best_tree.graphsummary[1]:
    print("Edge:",i[0],"-",i[1],"| Weight:",i[2])
    x += i[2]
print("Nodes in final tree",best_tree.V)
print("Minimum Cost",x)
print("Time taken to generate the Steiner Tree: %s seconds" % (time.time() - start_time))

Steiner Tree using Branch and Bound Algorithm with pruning:
Edges of the tree with corresponding weight
Edge: 4 - 5 | Weight: 3
Edge: 5 - 0 | Weight: 7
Edge: 3 - 5 | Weight: 1
Nodes in final tree [4, 5, 0, 3]
Minimum Cost 11
Time taken to generate the Steiner Tree: 0.036958932876586914 seconds
